# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 10 2023 9:52AM,259070,15,ALIFBA171TZVKZ4,"Alliance Pharma, Inc.",Released
1,Mar 10 2023 9:52AM,259070,15,ALIFBA171V0RKMK,"Alliance Pharma, Inc.",Released
2,Mar 10 2023 9:52AM,259070,15,ALIFBA171V2KZJ1,"Alliance Pharma, Inc.",Released
3,Mar 10 2023 9:52AM,259070,15,ALIFBA171TZVR4S,"Alliance Pharma, Inc.",Released
4,Mar 10 2023 9:52AM,259070,15,ALIFBA171V3V8XC,"Alliance Pharma, Inc.",Released
5,Mar 10 2023 9:52AM,259070,15,ALIFBA171V3YRX3,"Alliance Pharma, Inc.",Released
6,Mar 10 2023 9:44AM,259069,19,ADV80013589,"AdvaGen Pharma, Ltd",Released
7,Mar 10 2023 9:44AM,259069,19,ADV80013366,"AdvaGen Pharma, Ltd",Executing
8,Mar 10 2023 9:42AM,259068,10,37822,"Digital Brands, LLC",Released
9,Mar 10 2023 9:42AM,259068,10,37823,"Digital Brands, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,259067,Released,1
24,259068,Released,10
25,259069,Executing,1
26,259069,Released,1
27,259070,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259066,NaN,NaN,1.0
259067,NaN,NaN,1.0
259068,NaN,NaN,10.0
259069,NaN,1.0,1.0
259070,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258997,25.0,17.0,1.0
259011,0.0,0.0,1.0
259013,0.0,0.0,1.0
259014,0.0,0.0,1.0
259018,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258997,25,17,1
259011,0,0,1
259013,0,0,1
259014,0,0,1
259018,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258997,25,17,1
1,259011,0,0,1
2,259013,0,0,1
3,259014,0,0,1
4,259018,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258997,25,17,1
1,259011,,,1
2,259013,,,1
3,259014,,,1
4,259018,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 10 2023 9:52AM,259070,15,"Alliance Pharma, Inc."
6,Mar 10 2023 9:44AM,259069,19,"AdvaGen Pharma, Ltd"
8,Mar 10 2023 9:42AM,259068,10,"Digital Brands, LLC"
18,Mar 10 2023 9:40AM,259067,15,Lonza
19,Mar 10 2023 9:14AM,259066,10,Eye Pharma Inc
20,Mar 10 2023 9:09AM,259064,19,"Graystone, LLC"
21,Mar 10 2023 8:55AM,259061,10,Bio-PRF
27,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated"
28,Mar 10 2023 7:53AM,259058,19,"AdvaGen Pharma, Ltd"
35,Mar 10 2023 7:41AM,259057,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 10 2023 9:52AM,259070,15,"Alliance Pharma, Inc.",,,6
1,Mar 10 2023 9:44AM,259069,19,"AdvaGen Pharma, Ltd",,1,1
2,Mar 10 2023 9:42AM,259068,10,"Digital Brands, LLC",,,10
3,Mar 10 2023 9:40AM,259067,15,Lonza,,,1
4,Mar 10 2023 9:14AM,259066,10,Eye Pharma Inc,,,1
5,Mar 10 2023 9:09AM,259064,19,"Graystone, LLC",,,1
6,Mar 10 2023 8:55AM,259061,10,Bio-PRF,,1,5
7,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated",,,1
8,Mar 10 2023 7:53AM,259058,19,"AdvaGen Pharma, Ltd",,3,4
9,Mar 10 2023 7:41AM,259057,19,"AdvaGen Pharma, Ltd",,1,33


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 9:52AM,259070,15,"Alliance Pharma, Inc.",6,,
1,Mar 10 2023 9:44AM,259069,19,"AdvaGen Pharma, Ltd",1,1,
2,Mar 10 2023 9:42AM,259068,10,"Digital Brands, LLC",10,,
3,Mar 10 2023 9:40AM,259067,15,Lonza,1,,
4,Mar 10 2023 9:14AM,259066,10,Eye Pharma Inc,1,,
5,Mar 10 2023 9:09AM,259064,19,"Graystone, LLC",1,,
6,Mar 10 2023 8:55AM,259061,10,Bio-PRF,5,1,
7,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated",1,,
8,Mar 10 2023 7:53AM,259058,19,"AdvaGen Pharma, Ltd",4,3,
9,Mar 10 2023 7:41AM,259057,19,"AdvaGen Pharma, Ltd",33,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 9:52AM,259070,15,"Alliance Pharma, Inc.",6,,
1,Mar 10 2023 9:44AM,259069,19,"AdvaGen Pharma, Ltd",1,1,
2,Mar 10 2023 9:42AM,259068,10,"Digital Brands, LLC",10,,
3,Mar 10 2023 9:40AM,259067,15,Lonza,1,,
4,Mar 10 2023 9:14AM,259066,10,Eye Pharma Inc,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 9:52AM,259070,15,"Alliance Pharma, Inc.",6.0,NaN,NaN
1,Mar 10 2023 9:44AM,259069,19,"AdvaGen Pharma, Ltd",1.0,1.0,NaN
2,Mar 10 2023 9:42AM,259068,10,"Digital Brands, LLC",10.0,NaN,NaN
3,Mar 10 2023 9:40AM,259067,15,Lonza,1.0,NaN,NaN
4,Mar 10 2023 9:14AM,259066,10,Eye Pharma Inc,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 9:52AM,259070,15,"Alliance Pharma, Inc.",6.0,0.0,0.0
1,Mar 10 2023 9:44AM,259069,19,"AdvaGen Pharma, Ltd",1.0,1.0,0.0
2,Mar 10 2023 9:42AM,259068,10,"Digital Brands, LLC",10.0,0.0,0.0
3,Mar 10 2023 9:40AM,259067,15,Lonza,1.0,0.0,0.0
4,Mar 10 2023 9:14AM,259066,10,Eye Pharma Inc,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1554272,19.0,1.0,0.0
12,518037,1.0,1.0,0.0
15,777134,8.0,17.0,25.0
16,518102,1.0,1.0,0.0
19,1813358,68.0,5.0,0.0
21,518072,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1554272,19.0,1.0,0.0
1,12,518037,1.0,1.0,0.0
2,15,777134,8.0,17.0,25.0
3,16,518102,1.0,1.0,0.0
4,19,1813358,68.0,5.0,0.0
5,21,518072,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,19.0,1.0,0.0
1,12,1.0,1.0,0.0
2,15,8.0,17.0,25.0
3,16,1.0,1.0,0.0
4,19,68.0,5.0,0.0
5,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,19.0
1,12,Released,1.0
2,15,Released,8.0
3,16,Released,1.0
4,19,Released,68.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,0.0,0.0,25.0,0.0,0.0,0.0
Executing,1.0,1.0,17.0,1.0,5.0,0.0
Released,19.0,1.0,8.0,1.0,68.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,25.0,0.0,0.0,0.0
1,Executing,1.0,1.0,17.0,1.0,5.0,0.0
2,Released,19.0,1.0,8.0,1.0,68.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,25.0,0.0,0.0,0.0
1,Executing,1.0,1.0,17.0,1.0,5.0,0.0
2,Released,19.0,1.0,8.0,1.0,68.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()